# Aharonov–Bohm (AB) Effect and the X–θ Extension — A Clean, Testable Notebook

This notebook implements a numerically stable simulation of the magnetic Aharonov–Bohm (AB) phase and a minimal X–θ extension modeled as a second Abelian U(1)-like sector.

**Tip for VS Code users:** ensure *File → Preferences → Settings* has **Notebook › Markdown: Math** enabled. Use Markdown cells (not code cells) for the formulas below.

## AB phase oracle

The magnetic AB phase is

$$\Delta\varphi_{\rm AB} \,=\, \frac{q}{\hbar}\oint_{\mathcal C}\mathbf A\cdot d\boldsymbol\ell
 \,=\, \frac{q}{\hbar}\,\Phi \,=\, 2\pi\,\frac{\Phi}{\Phi_0},\qquad \Phi_0\equiv \frac{h}{|q|}. $$

**Plan**
1. Define constants and safe numerical helpers (arc-length weighted line integral).
2. Use a long-solenoid vector potential $\mathbf A = \dfrac{\Phi}{2\pi r}\,\hat{\boldsymbol\phi}$ for $r>R$.
3. Build two **open** interferometer arms $P_1, P_2$; compute phase by **open-path subtraction**:
$$\Delta\varphi = \frac{q}{\hbar}\Big(\int_{P_1}\mathbf A\cdot d\boldsymbol\ell - \int_{P_2}\mathbf A\cdot d\boldsymbol\ell\Big) 
= 2\pi\,\frac{\Phi}{\Phi_0}. $$
4. Run oracle, gauge-invariance, path-reversal, and resolution tests.
5. Add an internal X–θ “flux” contribution $\Delta\varphi_\theta=2\pi\,\Phi_\theta/\Phi_{\theta0}$ with $\Phi_{\theta0}=h/g_\theta$.

In [ ]:
# Section 1: Imports, constants, helpers
import numpy as np
import matplotlib.pyplot as plt
from math import pi

# Physical constants (SI)
h = 6.62607015e-34         # Planck constant [J⋅s]
hbar = h/(2*pi)             # reduced Planck [J⋅s]
qe = -1.602176634e-19       # electron charge [C] (negative)

def unitphi(x, y):
    """Return azimuthal unit vector \hat{phi} in 2D plane at (x,y).
    For r=0, return (0,0)."""
    r = np.hypot(x, y)
    if r == 0:
        return np.array([0.0, 0.0])
    # phi-hat = (-sinφ, cosφ) = (-y/r, x/r)
    return np.array([-y/r, x/r])

def Avec_long_solenoid(x, y, Phi=1.0, R=1.0):
    """Vector potential of an ideal long solenoid in Coulomb-like gauge for r>R:
    A = (Phi / (2π r)) ϕ-hat; inside r<R we set A=0 for simplicity (field confined).
    Units are generic; we only need line integrals to match flux.*
    *As usual in AB discussions, details of gauge inside do not affect the phase.
    """
    r = np.hypot(x, y)
    if r <= R:
        return np.array([0.0, 0.0])
    return (Phi/(2*pi*r)) * unitphi(x, y)

def line_integral_A_along_path(path_xy, Afunc):
    """Numerical line integral ∫ A·dl along a polyline path.
    path_xy: array of shape (N,2)
    Afunc:  callable (x,y)->(Ax,Ay)
    Uses midpoint rule per segment for stability.
    """
    p = np.asarray(path_xy)
    segs = p[1:] - p[:-1]
    mids = 0.5*(p[1:] + p[:-1])
    vals = []
    for (mx,my), (dx,dy) in zip(mids, segs):
        Ax, Ay = Afunc(mx, my)
        vals.append(Ax*dx + Ay*dy)
    return float(np.sum(vals))

def ab_phase_open_paths(P1, P2, Afunc, q=qe, hbar=hbar):
    """Open-path subtraction: Δφ = (q/ħ) [∫_{P1}A·dl − ∫_{P2}A·dl]."""
    i1 = line_integral_A_along_path(P1, Afunc)
    i2 = line_integral_A_along_path(P2, Afunc)
    return (q/hbar)*(i1 - i2)

def circle(center=(0,0), radius=3.0, n=2048, start_angle=0.0, stop_angle=2*np.pi):
    cx, cy = center
    t = np.linspace(start_angle, stop_angle, n)
    return np.c_[cx + radius*np.cos(t), cy + radius*np.sin(t)]


## Section 2: Build two open interferometer arms and test

We create two paths that start far left and end far right, one going *above* the solenoid and one going *below*. The open-path subtraction reconstructs the loop integral.

$$\Phi_0 = \frac{h}{|q|}\,,\qquad \frac{\Delta\varphi}{2\pi} = \frac{\Phi}{\Phi_0}. $$

In [ ]:
# Section 2: Construct two open paths P1 (upper) and P2 (lower)
def make_paths(x0=-10.0, x1=10.0, yoff=2.5, nseg=2000):
    # Piecewise linear: left->near-left, arc up or down, near-right->right
    xs = np.linspace(x0, x1, nseg)
    # Upper path: y = +yoff·tanh(...) for smooth detour
    yu = yoff*np.tanh(0.5*xs)
    # Lower path: y = -yoff·tanh(...)
    yl = -yu
    P1 = np.c_[xs, yu]
    P2 = np.c_[xs, yl]
    return P1, P2

Phi_true = 5.0   # choose a flux (arbitrary units) through solenoid of radius R
R = 1.0
P1, P2 = make_paths(yoff=3.0)

A = lambda x,y: Avec_long_solenoid(x, y, Phi=Phi_true, R=R)
dphi = ab_phase_open_paths(P1, P2, A, q=qe, hbar=hbar)

phi0 = h/abs(qe)  # flux quantum
pred_ratio = Phi_true/phi0
print("AB oracle check (units arbitrary):")
print(f"  Δφ/(2π) numeric  = {dphi/(2*np.pi):.6e}")
print(f"  Φ/Φ0 (expected) = {pred_ratio:.6e}")

In [ ]:
# Section 3: Plot geometry and a few field lines of A
xx = np.linspace(-6,6,32)
yy = np.linspace(-6,6,32)
XX, YY = np.meshgrid(xx, yy)
U = np.zeros_like(XX)
V = np.zeros_like(YY)
for i in range(XX.shape[0]):
    for j in range(XX.shape[1]):
        ax, ay = A(XX[i,j], YY[i,j])
        U[i,j] = ax
        V[i,j] = ay

fig = plt.figure(figsize=(6,6))
plt.streamplot(XX, YY, U, V, density=1.0)
plt.plot(P1[:,0], P1[:,1])
plt.plot(P2[:,0], P2[:,1])
circle_theta = np.linspace(0, 2*np.pi, 300)
plt.plot(R*np.cos(circle_theta), R*np.sin(circle_theta))
plt.axis('equal'); plt.title('Vector potential field lines and open paths')
plt.xlabel('x'); plt.ylabel('y')
plt.show()

## Section 4: Gauge and path tests
Basic sanity:
1. **Path reversal** flips the sign: $\Delta\varphi(P_2,P_1) = -\Delta\varphi(P_1,P_2)$.
2. **Refinement** (more segments) leaves the result unchanged within tolerance.
3. **Gauge shift** $\mathbf A\to\mathbf A + \nabla \chi$ leaves $\Delta\varphi$ invariant.

In [ ]:
# Path reversal test
dphi_rev = ab_phase_open_paths(P2, P1, A, q=qe, hbar=hbar)
print(f"Path reversal: dphi_rev/(2π) = {dphi_rev/(2*np.pi):.6e} (should be ≈ -dphi/(2π))")

# Refinement test
P1_fine, P2_fine = make_paths(yoff=3.0, nseg=8000)
dphi_fine = ab_phase_open_paths(P1_fine, P2_fine, A, q=qe, hbar=hbar)
print(f"Refinement delta: |fine - coarse|/(2π) = {abs(dphi_fine-dphi)/(2*np.pi):.3e}")

# Gauge-invariance test: add a pure gradient ∇χ to A
def grad_chi(x,y):
    # χ(x,y) = α x y  ⇒ ∇χ = (α y, α x)
    alpha = 0.1
    return np.array([alpha*y, alpha*x])

A_gauged = lambda x,y: A(x,y) + grad_chi(x,y)
dphi_g = ab_phase_open_paths(P1, P2, A_gauged, q=qe, hbar=hbar)
print(f"Gauge invariance: |gauged - original|/(2π) = {abs(dphi_g-dphi)/(2*np.pi):.3e}")

## Section 5: Minimal X–θ contribution
We model a second compact sector with an effective flux $\Phi_\theta$ and coupling $g_\theta$.
Define its “flux quantum” $\Phi_{\theta0}=h/g_\theta$, giving a phase
$$\Delta\varphi_\theta = 2\pi\,\frac{\Phi_\theta}{\Phi_{\theta0}} = 2\pi\,\frac{g_\theta\,\Phi_\theta}{h}. $$
The total phase is then $\Delta\varphi_{\rm tot}=\Delta\varphi_{\rm AB}+\Delta\varphi_\theta$.

In [ ]:
# X–θ sector helpers
def phi_theta_to_phase(phi_theta, g_theta):
    return 2*np.pi * (g_theta*phi_theta)/h

def total_phase(P1, P2, Afunc, q=qe, hbar=hbar, phi_theta=0.0, g_theta=1.0):
    return ab_phase_open_paths(P1, P2, Afunc, q=q, hbar=hbar) + phi_theta_to_phase(phi_theta, g_theta)

# Example: sweep an internal flux and observe linear phase shifts
phi_theta_list = np.linspace(0.0, 3.0, 7)  # arbitrary units
g_theta = 1.0
vals = [ total_phase(P1,P2,A, qe, hbar, pt, g_theta)/(2*np.pi) for pt in phi_theta_list ]
for pt, v in zip(phi_theta_list, vals):
    print(f"Φθ={pt:.2f}  →  Δφ_tot/(2π) = {v:.6e}")

## Section 6: Interference fringe shift example
For small phase differences near the screen, an idealized two-path intensity can be modeled as
$$ I(\Delta\varphi) \propto 1 + \cos(\Delta\varphi). $$
Plotting $I$ vs $\Phi/\Phi_0$ and vs $\Phi_\theta/\Phi_{\theta0}$ shows the independent controls.

In [ ]:
ratios = np.linspace(0, 3, 200)  # Φ/Φ0
dphi_ab = 2*np.pi*ratios
I_ab = 1 + np.cos(dphi_ab)

plt.figure()
plt.plot(ratios, I_ab)
plt.xlabel('Φ/Φ0'); plt.ylabel('Relative intensity I')
plt.title('Idealized two-path intensity vs magnetic flux ratio')
plt.show()

ratios_theta = np.linspace(0, 3, 200)  # Φθ/Φθ0
dphi_theta = 2*np.pi*ratios_theta
I_theta = 1 + np.cos(dphi_theta)

plt.figure()
plt.plot(ratios_theta, I_theta)
plt.xlabel('Φθ/Φθ0'); plt.ylabel('Relative intensity I')
plt.title('Idealized two-path intensity vs internal flux ratio')
plt.show()

## Notes on robust rendering of math in notebooks

1. Use **Markdown cells** for LaTeX, not code cells.
2. Prefer `$$ ... $$` for display math and `\( ... \)` for inline math.
3. In VS Code, enable **Notebook › Markdown: Math**. If formulas still fail, toggle the same setting and re-render the cell.
4. Avoid stray backslashes in normal text (they can escape characters). Use raw strings only in code.
5. Headings like `# Title` should not contain unbalanced math delimiters.
6. If you require LaTeX-quality fonts in plots, set `plt.rcParams['text.usetex']=True` *only* if a LaTeX distribution is installed.